In [1]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
from rdkit.Chem import Draw
import time
print(time.asctime())
from rdkit import rdBase
print(rdBase.rdkitVersion)
from rdkit.Chem import rdMolAlign
import py3Dmol
import os
from rdkit import RDConfig
from rdkit.Chem.FeatMaps import FeatMaps
fdef = AllChem.BuildFeatureFactory(os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef'))
print(fdef.GetFeatureFamilies())

from rdkit.Chem.Features.ShowFeats import _featColors as featColors
from rdkit.Chem import rdmolops


Wed Apr  3 13:29:22 2024
2020.09.3
('Donor', 'Acceptor', 'NegIonizable', 'PosIonizable', 'ZnBinder', 'Aromatic', 'Hydrophobe', 'LumpedHydrophobe')


In [2]:
smis = ['CC(OC(=O)c1c[nH]c2ccccc12)C1CCCCN1C',
 'CN1CCOc2c(C(=O)NC3CC4CCC(C3)N4C)cc(Cl)cc21',
 'CN1CC2CCC1CC2n1nnc2ccc(Cl)cc2c1=O']
ms = [Chem.MolFromSmiles(x) for x in smis]
Draw.MolsToGridImage(ms)

ms = [Chem.AddHs(m) for m in ms]
ps = AllChem.ETKDGv3()
ps.randomSeed = 0xf00d  # we seed the RNG so that this is reproducible

for m in ms:
    AllChem.EmbedMolecule(m,ps)
    
o3d = rdMolAlign.GetO3A(ms[1],ms[0])
o3d.Align()

0.4935300592533495

In [3]:
class molecule_over_molecule:
    def __init__(self, ms):
        self.ms = ms
        self.p = py3Dmol.view(width=400, height=400)

    def drawit(self, ms,p = None, confId=-1, removeHs=True,colors=('cyanCarbon','redCarbon','blueCarbon')):
            self.p.removeAllModels()

            if p is None:
                p = py3Dmol.view(width=400, height=400)
            p.removeAllModels()
            for i,m in enumerate(ms):
                if removeHs:
                    m = Chem.RemoveHs(m)
                IPythonConsole.addMolToView(m,p,confId=confId)
            for i,m in enumerate(ms):
                p.setStyle({'model':i,},
                                {'stick':{'colorscheme':colors[i%len(colors)]}})
            p.zoomTo()
            return p.show()
    

def drawit(m, feats, p=None, confId=-1, removeHs=True):
    self.p.removeAllModels()

    if p is None:
        p = py3Dmol.view(width=400, height=400)
    p.removeAllModels()
    if removeHs:
        m = Chem.RemoveHs(m)
    IPythonConsole.addMolToView(m,p,confId=confId)
    for feat in feats:
        pos = feat.GetPos()
        clr = featColors.get(feat.GetFamily(),(.5,.5,.5))
        p.addSphere({'center':{'x':pos.x,'y':pos.y,'z':pos.z},'radius':.5,'color':colorToHex(clr)});
    p.zoomTo()
    return p.show()

molecular_graph = molecule_over_molecule(ms)
molecular_graph.drawit(ms)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
keep = ('Donor','Acceptor','NegIonizable','PosIonizable','Aromatic')

def feats(ms, fdef):

    featLists = []
    for m in ms:
        rawFeats = fdef.GetFeaturesForMol(m)
    # filter that list down to only include the ones we're intereted in 
        featLists.append([f for f in rawFeats if f.GetFamily() in keep])
    return featLists

feats(ms,fdef )

[[<rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature at 0x2393e54a9e0>,

In [5]:
def colorToHex(rgb):
    rgb = [f'{int(255*x):x}' for x in rgb]
    return '0x'+''.join(rgb)
def drawit(m, feats, p=None, confId=-1, removeHs=True):
        if p is None:
            p = py3Dmol.view(width=400, height=400)
        p.removeAllModels()
        if removeHs:
            m = Chem.RemoveHs(m)
        IPythonConsole.addMolToView(m,p,confId=confId)
        for feat in feats:
            pos = feat.GetPos()
            clr = featColors.get(feat.GetFamily(),(.5,.5,.5))
            p.addSphere({'center':{'x':pos.x,'y':pos.y,'z':pos.z},'radius':.5,'color':colorToHex(clr)});
        p.zoomTo()
        return p.show()

In [6]:
keep = ('Donor','Acceptor','NegIonizable','PosIonizable','Aromatic')

class features_graph():
    '''First:
     - Find features of each molecule '''
    

    def __init__(self, ms, fdef):
        self.ms = ms
        self.fdef = fdef

    @staticmethod
    def feats(ms, fdef):
        featLists = []
        for m in ms:
            rawFeats = fdef.GetFeaturesForMol(m)
        # filter that list down to only include the ones we're intereted in 
            featLists.append([f for f in rawFeats if f.GetFamily() in keep])
        return featLists

    # feats(ms,fdef )

    @staticmethod
    def colorToHex(rgb):
        rgb = [f'{int(255*x):x}' for x in rgb]
        return '0x'+''.join(rgb)

    @staticmethod
    def drawit( m, feats, p=None, confId=-1, removeHs=True):
        # self.p.removeAllModels()

        if p is None:
            p = py3Dmol.view(width=400, height=400)
        p.removeAllModels()
        if removeHs:
            m = Chem.RemoveHs(m)
        IPythonConsole.addMolToView(m,p,confId=confId)
        for feat in feats:
            pos = feat.GetPos()
            clr = featColors.get(feat.GetFamily(),(.5,.5,.5))
            p.addSphere({'center':{'x':pos.x,'y':pos.y,'z':pos.z},'radius':.5,'color':colorToHex(clr)});
        p.zoomTo()
        return p.show()

mol = ms[0]
rdmolops.RemoveHs(mol, implicitOnly=False, updateExplicitCount=False, sanitize=True)
feature_graph = features_graph.feats(ms, fdef)
features_graph.drawit(mol, feature_graph[0])


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
mol = ms[0]
rdmolops.RemoveHs(mol, implicitOnly=False, updateExplicitCount=False, sanitize=True)

features_graph.drawit(ms[0], feature_graph[0])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
feature_graph

[[<rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature at 0x2393e569e40>,

In [9]:
def feats(ms, fdef):
        featLists = []
        for m in ms:
            rawFeats = fdef.GetFeaturesForMol(m)
        # filter that list down to only include the ones we're intereted in 
            featLists.append([f for f in rawFeats if f.GetFamily() in keep])
        return featLists

feats(ms,fdef)


[[<rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature at 0x2393e5a7ac0>,

In [10]:
fg = features_graph(ms, fdef)
feat_listss = features_graph.feats(ms, fdef)
features_graph.drawit(ms[0], feat_listss[0])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
fmParams = {}
for k in fdef.GetFeatureFamilies():
    fparams = FeatMaps.FeatMapParams()
    fmParams[k] = fparams

keep = ('Donor','Acceptor','NegIonizable','PosIonizable','Aromatic')
featLists = []
for m in ms:
    rawFeats = fdef.GetFeaturesForMol(m)
    # filter that list down to only include the ones we're intereted in 
    featLists.append([f for f in rawFeats if f.GetFamily() in keep])

In [12]:
drawit(ms[0],featLists[0])


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
drawit(ms[1],featLists[1])


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
drawit(ms[2],featLists[2])


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [15]:
fms = [FeatMaps.FeatMap(feats = x,weights=[1]*len(x),params=fmParams) for x in featLists]


In [16]:
def drawFeatMap(m, fMap, p=None, confId=-1, removeHs=True):
        if p is None:
            p = py3Dmol.view(width=400, height=400)
        p.removeAllModels()
        if removeHs:
            m = Chem.RemoveHs(m)
        IPythonConsole.addMolToView(m,p,confId=confId)
        for feat in fMap.GetFeatures():
            pos = feat.GetPos()
            clr = featColors.get(feat.GetFamily(),(.5,.5,.5))
            p.addSphere({'center':{'x':pos.x,'y':pos.y,'z':pos.z},'radius':feat.weight*.5,'color':colorToHex(clr)});
        p.zoomTo()
        return p.show()

In [17]:
drawFeatMap(ms[0],fms[0])


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol